# Fonction d'ordre supérieur

Les fonctions d'ordre supérieur dans Spark SQL sont des fonctions qui prennent d'autres fonctions comme arguments ou qui retournent des fonctions. Ces fonctions permettent d'effectuer des opérations complexes sur des données structurées telles que des tableaux ou des structures dans Spark SQL.

À partir de la version 2.4, Spark SQL a introduit plusieurs fonctions d'ordre supérieur pour travailler avec des données complexes. Voici quelques exemples de fonctions d'ordre supérieur couramment utilisées dans Spark SQL :

* **transform** : Applique une fonction donnée à chaque élément d'un tableau et retourne un nouveau tableau avec les résultats.<br />Syntaxe : `transform(array, function)`.
* **filter** : Retourne un nouveau tableau contenant les éléments qui satisfont la condition spécifiée par la fonction donnée.<br />Syntaxe : `filter(array, function)`.
* **exists** : Vérifie si au moins un élément d'un tableau satisfait la condition spécifiée par la fonction donnée.<br />Syntaxe : `exists(array, function)`.
* **aggregate** : Agrège les éléments d'un tableau à l'aide d'une fonction d'agrégation et d'une valeur initiale.<br />Syntaxe : `aggregate(array, initial_value, merge_function[, finish_function])`.

## Préambule

In [ ]:
import $ivy.`org.apache.spark::spark-core:3.3.2`
import $ivy.`org.apache.spark::spark-sql:3.3.2`
import $ivy.`org.slf4j:slf4j-reload4j:2.0.6`

import org.apache.logging.log4j.Level
import org.apache.logging.log4j.core.config.Configurator

// Avoid disturbing logs
Configurator.setRootLevel(Level.OFF)

In [ ]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.rdd._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    // L'appel ci-dessous sert à donner un nom à votre application
    // Ce apparaîtra notamment dans la Spark UI
    .appName("Sales Analysis - SparkSQL")
    .getOrCreate()
}

import spark.implicits._

// Ce script fournit que élément supplémentaires pour rendre l'affichage plus confortable
import $file.^.internal.spark_helper, spark_helper._

## Chargement

In [ ]:
val dataframe: DataFrame =
  spark.read
    // indique que le fichier contient une ligne d'en-tête qui servira
    // pour nommer les champs
    .option("header", true)
    // demande à Spark SQL de tenter de déterminer le type des colonnes
    .schema("id STRING, client STRING, timestamp TIMESTAMP, product STRING, price DOUBLE")
    // lecture du fichier au format CSV
    .csv("data/orders.csv")

import java.sql.Timestamp

case class Order(
  id:        String,
  clientId:  String,
  timestamp: Timestamp,
  product:   String,
  price:     Double
)

val orders: Dataset[Order] =
  dataframe
    .withColumnRenamed("client", "clientId")
    .as[Order]

orders.createOrReplaceTempView("orders")

In [ ]:
val pricesByDay =
  orders
    .groupBy(to_date($"timestamp").as("date"))
    .agg(collect_list($"price").as("price"))

pricesByDay.showHTML(truncate=120)

In [ ]:
pricesByDay
  .select(
      $"date",
      transform($"price", _ => lit(1)).as("one"),
      aggregate($"price", lit(0.0), _ + _).as("total")
  )

.showHTML()

In [ ]:
val result = spark.sql("""
SELECT
  date,
  transform(price, p -> 1) AS one,
  aggregate(price, CAST(0.0 AS double), (p1, p2) -> p1 + p2) AS total
FROM (
  SELECT
    to_date(timestamp) AS date,
    collect_list(price) AS price
  FROM orders
  GROUP BY date
)
""").showHTML(limit=10, truncate=120)

In [ ]:
orders.schema